# Tasks for BMS


##  Contents:

* [Custom PRS estimation in ALS](#custom)
  * [Create ALS cohort](#als-cohort)
    * [Modules and functions for LabKey](#labkey)
    * [Participants recruited for ALS](#recruited-als)
    * [Participants with ALS who were recruited for other diseases](#recruited-other)
    * [Make control cohort](#control)
    * [Get gVCF locations](#gvcf)
  * [Bring in PRS stats data](#prs-data)
  * [Import Docker image using Singularity](#singularity)
* [Germline variants in oncology cohorts](#germline-survival)
  * [Build cohorts of participants with particular germline variants](#germline-cohort)
  * [Getting data from LabKey for survival analysis](#survival-data)
* [Somatic variants association with survival](#somatic-survival)

##  Custom PRS estimation in ALS <a name="custom"></a>

Task:
* We would like to bring in our own data (PRS statistics) and apply to ALS cohort in GEL. These are currently in an S3 bucket in the format:  
  `phenotype chrom pos ref alt pvalue beta se t_stat nobs note`
* This would help outline to us the mechanisms of how to setup a container based (or similar environment) system to interact with external data. There is an existing container in Docker which uses gVCFs as input.
* This would also outline how to construct cohort using the right tables in labkey and extract genotypes for that cohort to estimate PRS

Input:
* PRS statistics table
* Phenotype

Output:
* Case-control cohort of participants with/without ALS with participant IDs and locations of gVCFs

### Create ALS cohort <a name="als-cohort"></a>

Creating cohorts was covered in a previous training session on [Building cohorts based on phenotypes](https://research-help.genomicsengland.co.uk/display/GERE/Building+a+cohort+based+on+phenotypes%2C+May+2022). Here we will summarise the relevant information from that to build ALS cohorts.

#### Modules and functions for Labkey <a name="labkey"></a>

This first section imports the relevant modules, defines a function `labkey_to_df` that queries labkey with the defined SQL query, and defines the version of the main programme to use - you should update then when a new data release comes out.

In [ ]:
import numpy as np
import functools
import labkey
import pandas as pd

def labkey_to_df(sql_query, database, maxrows):
    """generate an pandas dataframe from labkey sql query
    Args:
        sql_query (str): an sql query as string.
        database (str): The name of and path to the database as a string, 
            for example  "main-programme/main-programme_v15_2022-05-26"
        maxrows: the maximum number of rows to return
    """
    
    server_context = labkey.utils.create_server_context(
        domain = "labkey-embassy.gel.zone",
        container_path = database,
        context_path = "labkey",
        use_ssl = True
    )
    
    results =  labkey.query.execute_sql(
        server_context,
        schema_name = "lists",
        sql = sql_query,
        max_rows = maxrows
    )
    
    return(pd.DataFrame(results['rows']))

version = "main-programme/main-programme_v16_2022-10-13"

#### Participants recruited for ALS <a name="recruited-als"></a>

A number of participants were recruited to the rare disease programme for ALS and can be found with a simple query to the `rare_disease_participant_diseases` table. The following code fetches the participant IDs.

In [ ]:
disease = "Amyotrophic lateral sclerosis or motor neuron disease"

rd_sql = (f''' SELECT participant_id, specific_disease 
    FROM rare_diseases_participant_disease 
    WHERE rare_diseases_participant_disease.specific_disease = '{disease}'
    ''')

rd_query = labkey_to_df(rd_sql, version, 1000)
rd_query

#### Particpants with ALS who were recruited for other diseases <a name="recruited-other"></a>

To find additional individuals who have been enrolled for diseases other than ALS, but had an ICD-10 code indicative of ALS (G12.2) recorded in their hospital data, which can contain more recent visits, we can also query the `hes` tables.

In [ ]:
hes_tables = ["apc", "op"]
icd_code = "G122"

concatenated = []

for hes_table in hes_tables:
    sqlstr = (
        f'''
        SELECT participant_id, diag_all
        FROM hes_{hes_table}
        WHERE diag_all LIKE '%{icd_code}%'
        '''
    )
    query = labkey_to_df(sqlstr, version, 100000)
    concatenated += list(query['participant_id'])

We can now combine the two lists to get a full list of participants with ALS and filter to give only unique values.

In [ ]:
all_als = set(list(rd_query['participant_id']) + concatenated)
print ("Count: ", len(all_als), "\n", all_als)

#### Make control cohort <a name="control"></a>

We can get a group from the participant table and check its covariates to see if it matches.

In [ ]:
control_sql = (f'''
    SELECT participant_id, participant_phenotypic_sex, participant_ethnic_category, year_of_birth
    FROM participant
    WHERE participant_id NOT IN {*all_als,}
''')

control_query = labkey_to_df(control_sql, version, 1000)

control_list = list(control_query['participant_id'])

The MatchIt R algorithm is available to ensure matching between case and control cohorts.

#### Get gVCF locations <a name="gvcf"></a>

To get the gVCFs for PRS analysis, you can find the location of these using the `genome_file_paths_and_types` table. You can then incorporate the files into your pipeline.

In [ ]:
filetype = "Genomic VCF"

case_path_sql = (f'''
    SELECT participant_id, filename, file_path
    FROM genome_file_paths_and_types
    WHERE participant_id IN {*all_als,}
    AND file_sub_type = '{filetype}'
''')

case_path_query = labkey_to_df(case_path_sql, version, 100000)
case_path_query['case'] = "Yes"

control_path_sql = (f'''
    SELECT participant_id, filename, file_path
    FROM genome_file_paths_and_types
    WHERE participant_id IN {*control_list,}
    AND file_sub_type = '{filetype}'
''')

control_path_query = labkey_to_df(control_path_sql, version, 100000)
control_path_query['case'] = "No"

phenofile = pd.concat([case_path_query, control_path_query])
phenofile

### Bring in PRS stats data <a name="prs-data"></a>

There is no way to bring s3 buckets into RE1. You can add the data to the docker image, [bring it in using Airlock](https://research-help.genomicsengland.co.uk/display/GERE/How+to+use+the+Airlock+tool+to+import+and+export+files), or clone it in from Git on the HPC. This will be located in your secure df_BMS folder and nobody else will have access.

### Import Docker image using Singularity <a name="singularity"></a>

You can import your Docker image using Singularity, which is preinstalled on the HPC. There are instructions on using Singularity on the HPC [here](https://research-help.genomicsengland.co.uk/display/GERE/Using+containers+within+the+Research+Environment).

Start by logging into the HPC with:

`ssh <username>@corp.gel.ac@phpgridzlogn004.int.corp.gel.ac`

You will need to cd into your working directory:

`cd df_bms`

From here you can load Singularity:

`module load tools/singularity/3.8.3`

You can pull your docker image with singularity:

`singularity pull <docker image>`

This will create a singularity image in your working directory. You can now run the container with:

`singularity run <singularity container>`

##  Germline variants in oncology cohorts <a name="germline-survival"></a>

Task:
* Estimate _CTLA4_ and _FCGR_ specific germline variant effect on survival in cancer
* To our knowledge this would allow us to understand the mechanism to extract germline genotypes (that are not in the small subset of annotated risk factors) and to correlate this with possible response ICI treatment

### Build cohorts of participants with particular germline variants <a name="germline-cohort"></a>

We can use the [gene variant workflow](https://research-help.genomicsengland.co.uk/display/GERE/Gene-Variant+Workflow) to find all participants with variants in a gene.

1. Load copy the script as described in the documentation.
2. Edit the gene list to the two genes, and the submit_workflow.sh and variant_workflow_inputs.json files to have the correct project codes.
3. Submit the job as described in the documentation.

The results will appear in the folder `final_output/data`, in files named like `chr<chr>_<gene_name>_<Ensembl_ID>_<genome_assembly>_annotated_variants.tsv`.

There will be one table for GRCh37 and one for GRCh38. All of the cancer participants have their genomes aligned to GRCh38, so you can ignore the GRCh37 files.

This will not filter by rare disease vs cancer so we will need to use LabKey to pull out cancer participants. We'll start by making importing the table as a dataframe and pulling out the relevant data: I've gone for the location, alleles, consequences and samples. 

I'm also filtering to only get the consequence on the canonical transcript. I've used a further filter to only get variants with certain consequences. Edit the list `consequences` to find the consequences you're interested in.

In [ ]:
full_gene_variant_results = pd.read_csv(
    'genevariant/final_output/data/'
    'chr2_CTLA4_ENSG00000163599_GRCh38_annotated_variants.tsv' , 
    sep='\t')
consequences = [
    "missense_variant", 
    "transcript_ablation", 
    "splice_donor_variant", 
    "splice_acceptor_variant", 
    "frameshift_variant", 
    "stop_lost", 
    "start_lost", 
    "stop_gained", 
    "inframe_deletion", 
    "inframe_insertion"
    ]
full_gene_variant_results['in_list'] = (
    full_gene_variant_results
    .Consequence_annotation
    .apply(
        lambda x: any(i in x for i in consequences)
        )
    )
    
full_gene_variant_results
filtered_gv = full_gene_variant_results[
    (full_gene_variant_results['CANONICAL_annotation']=='YES') 
    & (full_gene_variant_results['in_list']==True)
    ]
short_gv = filtered_gv[[
    'ID_variant', 
    'REF_variant', 
    'ALT_variant', 
    'Consequence_annotation', 
    'Het_Samples', 
    'Hom_Samples']]
short_gv

Now we're going to wrangle so that there's one platekey per row.

In [ ]:
short_gv = pd.melt(
    short_gv, 
    id_vars = [
        'ID_variant',
        'REF_variant',
        'ALT_variant',
        'Consequence_annotation'
        ],
    value_vars = [
        'Het_Samples', 
        'Hom_Samples'
        ],
    var_name = 'zygosity',
    value_name = 'platekey'
    )

short_gv = short_gv.dropna()

short_gv['platekey'] = short_gv['platekey'].str.split(",")
short_gv = short_gv.explode('platekey')
short_gv

We can now look these Platekeys up in the Labkey cancer_analysis table, to filter to only cancer participants.

In [ ]:
platekeys = set(list(short_gv['platekey']))

cancer_sql = (
    f'''
    SELECT participant_id, germline_sample_platekey
    FROM cancer_analysis
    WHERE germline_sample_platekey IN  {*platekeys,}
    '''
)
cancer_query = labkey_to_df(cancer_sql, version, 100000)
cancer_query

### Getting data from LabKey for survival analysis <a name="survival-data"></a>

For survial analysis, you will need to find the birth, death and diagnosis dates of the participants. You may also wish to fetch covariate information such as sex, ethnicity and principal components. The following code fetches the dates for the list specified above.

In [ ]:
ctla4_part = set(list(cancer_query['participant_id']))

# import the python survival analysis functions:
import sys
sys.path.append('/pgen_int_work/BRS/christian/python_survival/')
import survival as su

# comparing on presence/absence of germline mutation in the entire 100K
# cancer cohort.
# you can ofcourse limit your cohort to, for example, only those who have
# received immune checkpoint blockade here.
sqlstr = ('''
    SELECT
        participant_id, disease_type
    FROM
        lists.cancer_analysis
    WHERE
        tumour_type = 'PRIMARY' ''')
ca = labkey_to_df(
    sqlstr,
    version, 
    100000)
    


initiate the survival class, through which we can extract:
- mortality: quer_ons()
- date of last follow up: quer_hes()
- date of diagnosis: quer_dod()
- match diagnosis date with cohort based on disease type: merge_dod()
- impute diagnosis date based on average per disease types for those with no diagnosis date: dod_impute()
- generate survival data from the above: surv_time()


In [ ]:
c = su.Survdat(
    ca, 
    ca['participant_id'],
    '/main-programme/main-programme_v16_2022-10-13',
    impute=False)
c.quer_ons()  # get mortality data : c.ons
c.quer_hes()  # query HES data for date of last follow up : c.hes
c.quer_dod()  # get date of diagnosis :c.dod
c.merge_dod()  # match date of diagnosis with cohort: c.pid_diag, c.no_diag
c.dod_impute()  # date of diagnosis from average per disease type c.full_diag
# impute is only appended to c.pid_diag if impute=True.
c.surv_time()

Now include the germline data, this can be multiple genes, each as its own True/False column.
With these columns a Group column can be made with the su.assign_groups()

This function assigns groups as follows:
'and' = Group 1 if all germline data is mutated (True)
'or' = Group 1 if any germline data is muated (True)
'full' = each combination of True/False for multiple genes is assigned its own group.


In [ ]:
c.surv_dat['ctla4'] = c.surv_dat['participant_id'].isin(ctla4_part)

grouped_surv_dat = su.assign_groups(
    dataframe=c.surv_dat,
    vars=['ctla4'],
    type='or'
    )
grouped_surv_dat

Finally plot the Kaplan-Meier curve:

In [ ]:
su.kmsurvival(
    data=grouped_surv_dat,
    strata=pd.unique(grouped_surv_dat['group']),
    output='./',
    plt_title='survival stratified by germline CTLA4 mutations',
    plot=True,
    table=True)

Covariate information is available in the `aggregate_gvcf_sample_stats` table. We're going to fetch sex, ethnicity and principal components for our list of participants.

In [ ]:
PCs = ["pc" + str(i+1) for i in range(20)]

cov_sql = (f''' SELECT participant_id,
    karyotype, {", ".join(str(x) for x in PCs)},
    pred_african_ancestries as AFR, 
    pred_south_asian_ancestries as SAS, 
    pred_east_asian_ancestries as EAS, 
    pred_european_ancestries as EUR, 
    pred_american_ancestries as AMR
    FROM aggregate_gvcf_sample_stats
    WHERE participant_id IN {*ctla4_part,}
    ''')

cov_query = labkey_to_df(cov_sql, version, 10000)
cov_query

We recommend using the genomically determined ethnicity to filter your cohort. As a general rule:
* If score ≥ 0.8, participant is this population
* If all scores <0.8, participant is admixed

The following code calculates the genomic ethnicity, following these rules.

In [ ]:
gen_eth = ["EUR", "SAS", "EAS", "AFR", "AMR"]

cov_query['max_eth'] = cov_query[gen_eth].max(axis="columns")
cov_query['max_eth_index'] = cov_query[gen_eth].idxmax(axis="columns")

cov_query['gen_eth'] = cov_query.apply(
    lambda x: 
        x['max_eth_index'] if x['max_eth'] >= 0.8 else 'Admixed',
     axis=1)
cov_query

You can use these data for survival analysis.

##  Somatic variants association with survival <a name="somatic-survival"></a>

Task
* Effect of somatic STK11 mutations on survival in NSCLC patients

This can be done using the [R survival analysis](https://research-help.genomicsengland.co.uk/display/GERE/Survival+-+cancer) script, which carries out survival analysis comparing participants with somatic variants in a particular gene to those without.

1. Load copy the script as described in the documentation.
2. Edit line 350 to your gene of interest
3. Run the script with R

The same can be achieved with the Python survival analysis script, set up the submit_surv.sh: 

python3 /pgen_int_work/BRS/christian/python_survival/survival.py \
    --genes STK11 \
    --disease LUNG \
    -s or \
    --imputate \
    -o ./surv_out/